## Oil and Gas Production Forecast
### Part_3.
### 1.1 Import Librries

In [1]:
import pandas as pd
import numpy as np
import warnings

from  datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import category_encoders as ce
import matplotlib.pyplot as plt
import gc
%matplotlib inline
warnings.filterwarnings('ignore')

### 1.2 Read in data

In [2]:
fld = pd.read_csv('data/fld.csv')

In [3]:
df_1 = pd.read_csv('data/df_1.csv')
df_2 = pd.read_csv('data/df_2.csv')
df_3 = pd.read_csv('data/df_3.csv')
df = df_1.append(df_2).append(df_3)
del df_1, df_2, df_3
gc.collect()

11

In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df.head(4).append(df.tail(3))

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
0,31009019120000,1988,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
1,31009019120000,1989,"Sager, Howard & Judith",0.0,500.0,0.0,0.0,GD
2,31009019120000,1990,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
3,31009019120000,1991,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
121155,31029255050000,2017,U. S. Energy Development Corporation,0.0,2126.0,42.0,7.0,Gas Development
121156,31029255050000,2018,U. S. Energy Development Corporation,0.0,318.0,45.0,7.0,Gas Development
121157,31029255050000,2019,U. S. Energy Development Corporation,0.0,2436.0,0.0,12.0,Gas Development


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121158 entries, 0 to 121157
Data columns (total 8 columns):
API Well Number         121158 non-null int64
Year                    121158 non-null int64
Operator                121158 non-null object
OIL (Bbls)              102545 non-null float64
GAS (Mcf)               120678 non-null float64
WATER (Bbls)            116452 non-null float64
Months in Production    120252 non-null float64
Well Type               121158 non-null object
dtypes: float64(4), int64(2), object(2)
memory usage: 7.4+ MB


In [7]:
fld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 51 columns):
API Well Number                   4717 non-null int64
County Code                       4717 non-null int64
API Hole Number                   4717 non-null int64
Sidetrack                         4717 non-null int64
Completion                        4717 non-null int64
Well Name                         4717 non-null object
Company Name                      4689 non-null object
Operator Number                   4717 non-null int64
Well Type                         4717 non-null object
Map Symbol                        4717 non-null object
Well Status                       4717 non-null object
Status Date                       572 non-null object
Permit Application Date           4413 non-null object
Permit Issued Date                4455 non-null object
Date Spudded                      4629 non-null object
Date of Total Depth               4460 non-null object
Date Well Completed 

In [8]:
fld.sample(3)

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Well Name,Company Name,Operator Number,Well Type,Map Symbol,...,Permit Fee,Objective Formation,Depth Fee,Spacing,Spacing Acres,Integration,Hearing Date,Date Last Modified,DEC Database Link,Location 1
3071,31013181090000,13,18109,0,0,"Carlson,O 1",PPP Future Development Inc.,2740,GD,GW,...,1100.0,Medina,1000.0,NaN,NaN,NaN,NaN,2017-10-13T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.17235', 'needs_recoding': F..."
1794,31013135460000,13,13546,0,0,"Alday,F 1",Empire Energy E&P LLC,2544,GD,GW,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2010-06-04T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.73659', 'needs_recoding': F..."
2031,31013144940000,13,14494,0,0,Thompson 581,Empire Energy E&P LLC,2544,GD,GW,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2010-06-04T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.56917', 'needs_recoding': F..."


### Data Cleaning

In [9]:
fld['Well Type'].unique()

array(['GD'], dtype=object)

In [10]:
fld['Map Symbol'].unique()

array(['GWP', 'GW', 'O'], dtype=object)

In [11]:
fld.describe()

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Operator Number,Region,"Proposed Depth, ft",Surface Longitude,Surface Latitude,Bottom Hole Longitude,Bottom Hole Latitude,"True Vertical Depth, ft","Measured Depth, ft","Kickoff, ft","Drilled Depth, ft","Elevation, ft",Permit Fee,Depth Fee,Spacing Acres
count,4.717000e+03,4717.000000,4717.000000,4717.0,4717.000000,4717.000000,4717.0,4716.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.0,4717.000000,4444.000000,4716.000000,4717.000000,22.000000
mean,3.101271e+13,12.544626,16953.453890,0.0,0.018868,2272.140767,9.0,2812.433418,-79.316538,42.258476,-79.316536,42.258484,3554.433326,3554.430570,0.0,3554.430570,1369.596985,611.036684,559.059360,53.965909
std,1.620988e+09,1.622432,6011.530453,0.0,0.146576,1527.626292,0.0,1690.720419,0.222828,0.143486,0.222827,0.143491,775.597958,775.597873,0.0,775.597873,345.668537,656.604968,612.319424,27.441319
min,3.100901e+13,9.000000,255.000000,0.0,0.000000,13.000000,9.0,0.000000,-79.761600,41.999480,-79.761600,41.999480,0.000000,0.000000,0.0,0.000000,3.000000,0.000000,0.000000,35.200000
25%,3.101311e+13,13.000000,13043.000000,0.0,0.000000,1192.000000,9.0,2100.000000,-79.509110,42.151120,-79.509110,42.151120,3116.000000,3116.000000,0.0,3116.000000,1260.000000,0.000000,0.000000,40.000000
50%,3.101316e+13,13.000000,17148.000000,0.0,0.000000,2544.000000,9.0,3520.000000,-79.293470,42.257070,-79.293470,42.257070,3708.000000,3708.000000,0.0,3708.000000,1461.000000,20.000000,0.000000,41.030000
75%,3.101319e+13,13.000000,20246.000000,0.0,0.000000,2544.000000,9.0,4095.000000,-79.139240,42.369260,-79.139240,42.369260,4166.000000,4166.000000,0.0,4166.000000,1610.000000,1225.000000,1125.000000,51.130000
max,3.102966e+13,29.000000,67321.000000,0.0,2.000000,9998.000000,9.0,6501.000000,-78.756970,42.693560,-78.756970,42.693560,6182.000000,6182.000000,0.0,6182.000000,2625.000000,11000.000000,10900.000000,146.800000


In [12]:
fld['Completion'].value_counts()

0    4635
1      75
2       7
Name: Completion, dtype: int64

In [13]:
# fld['Spacing Acres'].unique()

In [14]:
# fld['Location 1'].unique()

In [15]:
# fld['Surface Latitude'].unique()
fld['Original Well Type'].unique()

array(['NL', 'GW', 'GD', 'OD'], dtype=object)

In [16]:
# Drop Numerical columns which are not required for analysis
fld.drop(columns=['API Hole Number','Sidetrack','Permit Fee', 
                  'Depth Fee', 'Spacing Acres'], inplace=True)

In [17]:
# Drop Categorical columns which are not required for analysis
fld.drop(columns=['Financial Security','Location 1', 'Integration',
                  'Hearing Date', 'Date Last Modified', 'DEC Database Link',
                  'Spacing', 'State Lease','Well Name'], inplace=True)

In [18]:
fld["Status Date"].isna().sum()

4145

In [19]:
fld['Company Name'].nunique(), fld['Operator Number'].nunique()

(352, 353)

In [20]:
fld.drop(columns=['Status Date', 'Permit Application Date', 'Map Symbol',
                 'Permit Issued Date','Company Name'], inplace=True)

In [21]:
fld.Quad.unique()

array(['Cherry Creek', 'Collins Center', 'Gowanda', 'Steamburg',
       'Farnham', 'North Collins', 'Randolph', 'New Albion', 'Kennedy',
       'Ivory', 'Perrysburg', 'Cattaraugus', 'Brocton', 'Silver Creek',
       'Dunkirk', 'Forestville', 'Cassadaga', 'Ellery Center',
       'North Of Dunkirk', 'Gerry', 'Ripley', 'South Ripley', 'Westfield',
       'Chautauqua', 'Hartfield', 'Jamestown', 'Sherman', 'Panama',
       'Clymer', 'North Clymer', 'Hamlet', 'Lakewood', 'North East PA',
       'North East', 'Wattsburg, PA-NY', 'Sugar Grove, PA-NY',
       'Warren Pa', 'North Of Westfield', 'Youngsville PA', 'Langford',
       'Angola'], dtype=object)

In [22]:
fld['Quad Section'].unique()

array(['I', 'G', 'F', nan, 'C', 'D', 'H', 'B', 'A', 'E'], dtype=object)

In [23]:
fld['Quad Section'].isna().sum()

9

In [24]:
fld['Date Well Confidentiality Ends'].isna().sum()

3090

In [25]:
#fld['Confidentiality Code'].isna().sum(), 
fld['Confidentiality Code'].unique()

array(['Pre-1989 Well (N/A)', 'Released'], dtype=object)

In [26]:
del fld['Date Well Confidentiality Ends'], fld['Confidentiality Code'], fld['Town']

In [27]:
fld[['Date Spudded','Date of Total Depth','Date Well Completed', 'Date Well Plugged']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 4 columns):
Date Spudded           4629 non-null object
Date of Total Depth    4460 non-null object
Date Well Completed    4717 non-null object
Date Well Plugged      353 non-null object
dtypes: object(4)
memory usage: 147.5+ KB


In [28]:
del fld['Date Spudded'], fld['Date of Total Depth'],fld['Date Well Plugged']

In [29]:
fld.columns

Index(['API Well Number', 'County Code', 'Completion', 'Operator Number',
       'Well Type', 'Well Status ', 'Date Well Completed', 'Quad',
       'Quad Section', 'Producing Field', 'Producing Formation', 'Slant',
       'County', 'Region', 'Proposed Depth, ft', 'Surface Longitude',
       'Surface Latitude', 'Bottom Hole Longitude', 'Bottom Hole Latitude',
       'True Vertical Depth, ft', 'Measured Depth, ft', 'Kickoff, ft',
       'Drilled Depth, ft', 'Elevation, ft', 'Original Well Type',
       'Objective Formation'],
      dtype='object')

In [30]:
fld['Date Well Completed'] = pd.to_datetime(fld['Date Well Completed'])
fld ['Completion year'] = fld['Date Well Completed'].dt.year
fld ['Completion month'] = fld['Date Well Completed'].dt.month
fld ['Completion day'] = fld['Date Well Completed'].dt.day
del fld['Date Well Completed']
# calendar['week'] = calendar['date'].dt.week.astype(np.int8)

In [31]:
# fld.isna().sum()

In [32]:
fld['Objective Formation'].unique()

array([nan, 'Medina', 'Onondaga', 'Clinton', 'Bass Island', 'Glade',
       'Theresa', 'Whirlpool', 'Queenston'], dtype=object)

In [33]:
fld['Producing Formation'].unique()

array(['Medina', nan, 'Whirlpool', 'Onondaga', 'Grimsby', 'Camillus',
       'Upper Devonian', 'Unknown', 'Salina', 'Bass Island', 'Tully',
       'Upper Devonian Shale'], dtype=object)

In [34]:
# drop Ob`jective formation, and leave Producing formation as actual one
del fld['Objective Formation']

In [35]:
fld[fld.isna().any(axis=1)]

,API Well Number,County Code,Completion,Operator Number,Well Type,Well Status,Quad,Quad Section,Producing Field,Producing Formation,...,Bottom Hole Latitude,"True Vertical Depth, ft","Measured Depth, ft","Kickoff, ft","Drilled Depth, ft","Elevation, ft",Original Well Type,Completion year,Completion month,Completion day
9,31009011610000,9,0,261,GD,PA,Steamburg,NaN,Lakeshore,Medina,...,42.11175,4765.0,4765.0,0.0,4765.0,1760.0,NL,1937,2,1
11,31009016650000,9,0,16,GD,PA,Gowanda,NaN,Lakeshore,Medina,...,42.38840,3413.0,3413.0,0.0,3413.0,1372.0,NL,1911,12,10
20,31009128650000,9,0,460,GD,AC,Farnham,H,Lakeshore,Medina,...,42.50212,2435.0,2435.0,0.0,2435.0,NaN,NL,1977,10,10
25,31009133630000,9,0,826,GD,PA,Gowanda,F,Lakeshore,Medina,...,42.42374,3046.0,3046.0,0.0,3046.0,NaN,NL,1934,5,22
26,31009133640000,9,0,431,GD,PA,Gowanda,F,Lakeshore,Medina,...,42.42113,2985.0,2985.0,0.0,2985.0,NaN,NL,1934,6,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702,31029147150000,29,0,9006,GD,UN,Gowanda,B,Lakeshore,Medina,...,42.48012,2359.0,2359.0,0.0,2359.0,NaN,NL,1927,1,2
4703,31029147170000,29,0,9006,GD,UN,North Collins,G,Lakeshore,Medina,...,42.51051,2340.0,2340.0,0.0,2340.0,NaN,NL,1926,9,30
4714,31029557810000,29,0,2470,GD,PA,Angola,E,Lakeshore,NaN,...,42.69356,1500.0,1500.0,0.0,1500.0,600.0,NL,2009,2,25
4715,31029664360000,29,0,9006,GD,UN,North Collins,G,Lakeshore,Medina,...,42.51828,2307.0,2307.0,0.0,2307.0,NaN,GD,1917,1,1


In [36]:
fld.isna().sum()

API Well Number              0
County Code                  0
Completion                   0
Operator Number              0
Well Type                    0
Well Status                  0
Quad                         0
Quad Section                 9
Producing Field              0
Producing Formation         15
Slant                        0
County                       0
Region                       0
Proposed Depth, ft           1
Surface Longitude            0
Surface Latitude             0
Bottom Hole Longitude        0
Bottom Hole Latitude         0
True Vertical Depth, ft      0
Measured Depth, ft           0
Kickoff, ft                  0
Drilled Depth, ft            0
Elevation, ft              273
Original Well Type           0
Completion year              0
Completion month             0
Completion day               0
dtype: int64

### GAS production for each well

In [37]:
df.sample(10)

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
109599,31013218450000,2006,Seneca Resources Corporation,NaN,267.0,0.0,12.0,Gas Development
46571,31013137300000,2018,U. S. Energy Development Corporation,0.0,3450.0,0.0,12.0,Gas Development
40776,31013125550000,2006,"Cotton Well Drilling Company, Inc.",NaN,1315.0,0.0,12.0,Gas Development
87825,31013183490000,2007,Dannic Energy Corp.,0.0,0.0,0.0,12.0,Gas Development
119020,31013248730001,2014,U. S. Energy Development Corporation,0.0,1182.0,60.0,12.0,Gas Development
93273,31013187460000,2012,Empire Energy E&P LLC,NaN,989.0,14.0,12.0,Gas Development
97566,31013191610000,1993,Vineyard Oil & Gas Company,0.0,3436.0,110.0,12.0,GD
90756,31013185500000,2011,"Morris, Geraldine M.",0.0,0.0,0.0,12.0,Gas Development
55863,31013148470000,2015,"Chautauqua Energy, Inc.",0.0,175.0,0.0,12.0,Gas Development
65467,31013159660000,2009,Dannic Energy Corp.,0.0,234.0,0.0,12.0,Gas Development


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121158 entries, 0 to 121157
Data columns (total 8 columns):
API Well Number         121158 non-null int64
Year                    121158 non-null int64
Operator                121158 non-null object
OIL (Bbls)              102545 non-null float64
GAS (Mcf)               120678 non-null float64
WATER (Bbls)            116452 non-null float64
Months in Production    120252 non-null float64
Well Type               121158 non-null object
dtypes: float64(4), int64(2), object(2)
memory usage: 7.4+ MB


In [39]:
df['Well Type'].unique()

array(['GD', 'Dry Hole', 'Gas Development', 'Not Listed', 'OD',
       'Gas Wildcat', 'GAS', 'G', 'Oil Development'], dtype=object)

In [40]:
df['Well Type'].value_counts()

Gas Development    75921
GD                 37982
Not Listed          7157
Dry Hole              42
OD                    38
Gas Wildcat            9
G                      7
Oil Development        1
GAS                    1
Name: Well Type, dtype: int64

In [41]:
df[df['Well Type']=='Gas Wildcat']

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
8897,31009187620000,2000,"Mosher, Victoria N.",NaN,0.00,NaN,12.0,Gas Wildcat
8898,31009187620000,2001,"Mosher, Victoria N.",0.0,300.00,0.0,12.0,Gas Wildcat
8899,31009187620000,2002,"Mosher, Victoria N.",0.0,300.00,0.0,1.0,Gas Wildcat
40533,31013125380000,2000,"Kastle Resources Enterprises, Inc.",NaN,496.87,NaN,9.0,Gas Wildcat
40534,31013125380000,2001,"Kastle Resources Enterprises, Inc.",NaN,3761.13,140.0,12.0,Gas Wildcat
40535,31013125380000,2002,"Kastle Resources Enterprises, Inc.",NaN,5796.14,35.0,NaN,Gas Wildcat
62797,31013157430000,2000,"Kastle Resources Enterprises, Inc.",NaN,952.67,NaN,9.0,Gas Wildcat
62798,31013157430000,2001,"Kastle Resources Enterprises, Inc.",NaN,5309.33,150.0,12.0,Gas Wildcat
62799,31013157430000,2002,"Kastle Resources Enterprises, Inc.",NaN,10793.65,8.0,NaN,Gas Wildcat


In [42]:
df[df['Well Type']=='OD']

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
2753,31009169280000,1990,"Keystone Energy Oil & Gas, Inc",0.0,0.0,0.0,0.0,OD
8858,31009187600000,1993,"Wood, Robert H. & Joyce",0.0,0.0,0.0,12.0,OD
22426,31013110980000,1994,Belden & Blake Corporation,0.0,1888.0,15.0,12.0,OD
22427,31013110980000,1995,Belden & Blake Corporation,0.0,1656.0,14.0,12.0,OD
22429,31013110980000,1997,Belden & Blake Corporation,0.0,1440.0,0.0,12.0,OD
22430,31013110980000,1998,Belden & Blake Corporation,0.0,1314.0,0.0,12.0,OD
22431,31013110980000,1999,Belden & Blake Corporation,0.0,1489.0,0.0,12.0,OD
38705,31013123620000,1988,"Resource Exploration, Inc.",553.0,4730.0,0.0,7.0,OD
38708,31013123620000,1991,"Resource America, Inc.",0.0,5270.0,728.0,12.0,OD
41075,31013125870000,1999,"Stedman Energy, Inc.",0.0,1875.0,115.0,12.0,OD


In [43]:
df[df['Well Type']=='Dry Hole']

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
9,31009019120000,2000,"Sager, Howard & Judith",NaN,0.0,NaN,12.0,Dry Hole
10,31009019120000,2001,"Sager, Howard & Judith",0.0,0.0,0.0,12.0,Dry Hole
11,31009019120000,2002,"Sager, Howard & Judith",0.0,NaN,0.0,12.0,Dry Hole
12,31009019120000,2003,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
13,31009019120000,2004,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
14,31009019120000,2005,"Sager, Howard & Judith",NaN,0.0,0.0,12.0,Dry Hole
15,31009019120000,2006,"Sager, Howard & Judith",NaN,0.0,0.0,12.0,Dry Hole
16,31009019120000,2007,"Sager, Howard & Judith",0.0,0.0,0.0,12.0,Dry Hole
17,31009019120000,2008,"Sager, Howard & Judith",0.0,0.0,0.0,12.0,Dry Hole
18,31009019120000,2009,"Sager, Howard & Judith",0.0,0.0,0.0,0.0,Dry Hole


In [44]:
df['GAS (Mcf)'].isna().sum()

480

In [45]:
df[df.isna().any(axis=1)]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
9,31009019120000,2000,"Sager, Howard & Judith",NaN,0.0,NaN,12.0,Dry Hole
11,31009019120000,2002,"Sager, Howard & Judith",0.0,NaN,0.0,12.0,Dry Hole
12,31009019120000,2003,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
13,31009019120000,2004,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
14,31009019120000,2005,"Sager, Howard & Judith",NaN,0.0,0.0,12.0,Dry Hole
...,...,...,...,...,...,...,...,...
121087,31029247590000,2007,U. S. Energy Development Corporation,NaN,1770.0,100.0,3.0,Not Listed
121088,31029247590000,2008,U. S. Energy Development Corporation,NaN,6301.0,75.0,11.0,Gas Development
121100,31029250720000,2008,U. S. Energy Development Corporation,NaN,5050.0,80.0,8.0,Gas Development
121112,31029250730000,2008,U. S. Energy Development Corporation,NaN,4852.0,60.0,5.0,Gas Development


In [46]:
df[df['GAS (Mcf)'].isna()]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
11,31009019120000,2002,"Sager, Howard & Judith",0.0,NaN,0.0,12.0,Dry Hole
12,31009019120000,2003,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
13,31009019120000,2004,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
109,31009128650000,2002,"Guzzetta, Joseph & Carlyn",0.0,NaN,0.0,12.0,Gas Development
110,31009128650000,2003,"Guzzetta, Joseph & Carlyn",NaN,NaN,NaN,12.0,Gas Development
...,...,...,...,...,...,...,...,...
115068,31013234460000,2000,U. S. Energy Development Corporation,NaN,NaN,NaN,NaN,Gas Development
115069,31013234460000,2001,U. S. Energy Development Corporation,NaN,NaN,NaN,NaN,Not Listed
115087,31013234480000,2000,"Goose Creek Drilling, Inc.",NaN,NaN,NaN,NaN,Gas Development
115124,31013234950000,2000,"Universal Resources Holdings, Incorporated",NaN,NaN,NaN,NaN,Gas Development


In [47]:
df[df['API Well Number']==31009019120000]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
0,31009019120000,1988,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
1,31009019120000,1989,"Sager, Howard & Judith",0.0,500.0,0.0,0.0,GD
2,31009019120000,1990,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
3,31009019120000,1991,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
4,31009019120000,1992,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
5,31009019120000,1993,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
6,31009019120000,1994,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
7,31009019120000,1998,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
8,31009019120000,1999,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
9,31009019120000,2000,"Sager, Howard & Judith",NaN,0.0,NaN,12.0,Dry Hole


In [48]:
df[df['API Well Number']==31013234460000]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
115068,31013234460000,2000,U. S. Energy Development Corporation,NaN,NaN,NaN,NaN,Gas Development
115069,31013234460000,2001,U. S. Energy Development Corporation,NaN,NaN,NaN,NaN,Not Listed
115070,31013234460000,2002,U. S. Energy Development Corporation,0.0,25525.0,4.0,12.0,Gas Development
115071,31013234460000,2003,Great Lakes Energy Partners,0.0,13023.0,0.0,12.0,Gas Development
115072,31013234460000,2004,Great Lakes Energy Partners,0.0,9857.0,0.0,12.0,Gas Development
115073,31013234460000,2005,Great Lakes Energy Partners,0.0,7737.0,0.0,12.0,Gas Development
115074,31013234460000,2006,Great Lakes Energy Partners,0.0,7098.0,0.0,12.0,Gas Development
115075,31013234460000,2007,Great Lakes Energy Partners,0.0,6273.0,0.0,12.0,Gas Development
115076,31013234460000,2008,"Range Resources-Appalachia, LLC",0.0,5982.0,0.0,12.0,Gas Development
115077,31013234460000,2009,"Range Resources-Appalachia, LLC",0.0,4766.0,0.0,12.0,Gas Development


In [49]:
df[df['API Well Number']==31009128650000]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
94,31009128650000,1987,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,Not Listed
95,31009128650000,1988,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,GD
96,31009128650000,1989,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,GD
97,31009128650000,1990,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,GD
98,31009128650000,1991,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,Not Listed
99,31009128650000,1992,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,0.0,GD
100,31009128650000,1993,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,12.0,GD
101,31009128650000,1994,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,12.0,GD
102,31009128650000,1995,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,12.0,GD
103,31009128650000,1996,"Guzzetta, Joseph & Carlyn",0.0,0.0,0.0,12.0,GD


In [50]:
df['API Well Number'].nunique()

4025

In [51]:
fld['API Well Number'].nunique()

4717

In [52]:
df[df['OIL (Bbls)'].isna()]

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
9,31009019120000,2000,"Sager, Howard & Judith",NaN,0.0,NaN,12.0,Dry Hole
12,31009019120000,2003,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
13,31009019120000,2004,"Sager, Howard & Judith",NaN,NaN,NaN,12.0,Dry Hole
14,31009019120000,2005,"Sager, Howard & Judith",NaN,0.0,0.0,12.0,Dry Hole
15,31009019120000,2006,"Sager, Howard & Judith",NaN,0.0,0.0,12.0,Dry Hole
...,...,...,...,...,...,...,...,...
121087,31029247590000,2007,U. S. Energy Development Corporation,NaN,1770.0,100.0,3.0,Not Listed
121088,31029247590000,2008,U. S. Energy Development Corporation,NaN,6301.0,75.0,11.0,Gas Development
121100,31029250720000,2008,U. S. Energy Development Corporation,NaN,5050.0,80.0,8.0,Gas Development
121112,31029250730000,2008,U. S. Energy Development Corporation,NaN,4852.0,60.0,5.0,Gas Development


In [53]:
# df[df['API Well Number']==31029247590000]

In [54]:
df.describe()

,API Well Number,Year,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production
count,1.211580e+05,121158.000000,102545.000000,120678.000000,116452.000000,120252.000000
mean,3.101271e+13,2003.428977,4.753925,2279.749555,28.436260,10.108248
std,1.389741e+09,9.821654,254.155124,4840.363613,392.594445,8.727938
min,3.100902e+13,1966.000000,0.000000,-16.000000,0.000000,0.000000
25%,3.101312e+13,1995.000000,0.000000,397.000000,0.000000,10.000000
50%,3.101315e+13,2004.000000,0.000000,1184.000000,0.000000,12.000000
75%,3.101319e+13,2012.000000,0.000000,2481.000000,6.000000,12.000000
max,3.102926e+13,2019.000000,61710.000000,269741.000000,55834.000000,370.000000


In [55]:
df['Well Type'].isna().any()

False

In [56]:
fld['Completion year'].value_counts()

1982    482
1983    412
1984    373
1981    308
1975    280
       ... 
2012      1
1954      1
1969      1
1912      1
1955      1
Name: Completion year, Length: 91, dtype: int64

In [69]:
# redefine field data frame with wells where gas production available
fld = fld.loc[fld['API Well Number'].isin(df['API Well Number'].unique())]
fld.reset_index(drop=True, inplace=True)

In [71]:
fld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4025 entries, 0 to 4024
Data columns (total 27 columns):
API Well Number            4025 non-null int64
County Code                4025 non-null int64
Completion                 4025 non-null int64
Operator Number            4025 non-null int64
Well Type                  4025 non-null object
Well Status                4025 non-null object
Quad                       4025 non-null object
Quad Section               4023 non-null object
Producing Field            4025 non-null object
Producing Formation        4025 non-null object
Slant                      4025 non-null object
County                     4025 non-null object
Region                     4025 non-null float64
Proposed Depth, ft         4025 non-null float64
Surface Longitude          4025 non-null float64
Surface Latitude           4025 non-null float64
Bottom Hole Longitude      4025 non-null float64
Bottom Hole Latitude       4025 non-null float64
True Vertical Depth, ft    